### 以台灣家戶收支調查資料復現 Efficient Responses to Targeted Cash Transfers (Attanasio & Lechene, 2014)
2025-spring 勞動經濟學 期末報告 \
台大經濟系 陳彥廷 B11303039 \
可以透過下方 Google Docs 連結看到實驗的設計、架構與結果分析等等 \
https://docs.google.com/document/d/11R1aUdc4legB_VUC_Wl2dtdAOrazGcdh4_wAkX1U-TQ/edit?usp=sharing


### install & import

In [4]:
!pip install pandas numpy matplotlib statsmodels linearmodels
# 資料處理
import pandas as pd
import numpy as np

# 傾向分數匹配（PSM）
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler
from sklearn.neighbors import NearestNeighbors

# 迴歸分析＆ Wald 檢定
import statsmodels.api as sm
import statsmodels.formula.api as smf

# 如果要做系統方程 （SUR）估計
from linearmodels.system import SUR
# 畫圖
import matplotlib.pyplot as plt

### 讀檔案、Pre-processing
篩選出「家戶內只有戶長與配偶在賺錢」+「過去一年有買過彩券」\
再透過年齡、教育程度、所得進行分組

In [5]:
df = pd.read_stata('Data/104 家庭收支調查/inc104.dta')
print(len(df))

16528


In [ ]:
# step 1: extract data of 戶長、配偶
data = []
data_columns = ["戶長性別", "戶長年齡", "戶長教育程度別", "戶長所得收入", "戶長彩券收入", \
         "配偶性別", "配偶年齡", "配偶教育程度別", "配偶所得收入", "配偶彩券收入", \
         "戶長性別", "戶內人數", "主食品支出", "副食品支出", "乳酪及蛋類", "水果類支出"]  # 非耐久財支出

for idx, row in df[(df["a9"] == 2) & (df["itm890"] > 0)].iterrows():  # 家戶內只有兩個就業人口 + 有買過彩券
    temp_data = []

    # 加入每個人的 性別、年齡、教育程度別、所得收入、彩券收入  // 人身意外災害保險、經常移轉收入 (按順序)
    for i in range(1, 9):
        if row[f"b2_{i}"] == '本人' and row[f"itm40{i}"] > 0:  # 本人
            temp_data += [row[f"b3_{i}"], row[f"b4_{i}"], row[f"b5_{i}"], row[f"itm40{i}"], row[f"itm91{i}"]] # , row[f"itm46{i}"] , row[f"itm41{i}"]]
    for i in range(1, 9):
        if row[f"b2_{i}"] == '配偶' and row[f"itm40{i}"] > 0:  # 配偶
            temp_data += [row[f"b3_{i}"], row[f"b4_{i}"], row[f"b5_{i}"], row[f"itm40{i}"], row[f"itm91{i}"]] # , row[f"itm46{i}"] , row[f"itm41{i}"]]
    
    # 有抓到本人和配偶 -> 加入 戶長性別、戶內人數、主食品支出、副食品支出、乳酪及蛋類、水果類支出
    if len(temp_data) == 10:  
        temp_data += [row["a7"], row["a8"], row["itm1001"], row["itm1002"], row["itm1003"], row["itm1004"]] 
        data.append(temp_data)

new_df = pd.DataFrame(data, columns = data_columns)
new_df['x'] = new_df[["主食品支出", "副食品支出", "乳酪及蛋類", "水果類支出"]].sum(axis=1)
new_df['log_x']  = np.log(new_df['x'])
new_df['log_x2'] = new_df['log_x']**2
print(len(new_df))
print(new_df.isna().sum())
new_df = new_df.fillna(0)


946
戶長性別         0
戶長年齡         0
戶長教育程度別      0
戶長所得收入       0
戶長彩券收入     713
配偶性別         0
配偶年齡         0
配偶教育程度別      0
配偶所得收入       0
配偶彩券收入     834
戶長性別         0
戶內人數         0
主食品支出        0
副食品支出       11
乳酪及蛋類        0
水果類支出        0
收入           0
x            0
log_x        0
log_x2       0
dtype: int64


In [12]:
from sklearn.preprocessing import StandardScaler
from sklearn.cluster import KMeans

# 1. 選取要分群的特徵（數值＋編碼後的類別變數）
features = new_df[[
    "戶長年齡", "戶長教育程度別", "戶長所得收入",
    "配偶年齡", "配偶教育程度別", "配偶所得收入"
]]

# 2. 類別變數 one-hot 編碼
features = pd.get_dummies(features,
    columns=["戶長教育程度別", "配偶教育程度別"],
    drop_first=True  # 去掉一欄，避免多重共線性
)

# 3. 標準化
scaler = StandardScaler()
X_scaled = scaler.fit_transform(features)

# 4. 建立並訓練 K-Means 模型
kmeans = KMeans(n_clusters=5, random_state=42)
new_df["cluster"] = kmeans.fit_predict(X_scaled)

# 5. 檢視各群大小
print(new_df["cluster"].value_counts())


cluster
2    274
3    251
1    187
4    159
0     75
Name: count, dtype: int64


In [18]:
# 對每一群做基本的分析(看不懂)
for i in range(5):
    test = new_df[new_df["cluster"] == i]
    print("-"*20, i, "-"*20)
    print(test.describe())

-------------------- 0 --------------------
            戶長年齡        戶長所得收入       戶長彩券收入       配偶年齡        配偶所得收入  \
count  75.000000  7.500000e+01    75.000000  75.000000  7.500000e+01   
mean   48.960000  7.132594e+05   245.333333  46.333333  5.106306e+05   
std     9.490308  3.479970e+05   611.222376  10.105301  3.066525e+05   
min    26.000000  1.531610e+05     0.000000  23.000000  1.158070e+05   
25%    44.500000  5.228310e+05     0.000000  40.000000  3.146910e+05   
50%    50.000000  6.608690e+05     0.000000  48.000000  4.359440e+05   
75%    55.000000  8.389345e+05   100.000000  53.000000  6.796425e+05   
max    68.000000  2.425809e+06  3000.000000  66.000000  2.235237e+06   

            配偶彩券收入       戶內人數         主食品支出          副食品支出         乳酪及蛋類  \
count    75.000000  75.000000     75.000000      75.000000     75.000000   
mean     81.333333   3.826667  17061.466667   61894.133333  10820.400000   
std     284.601824   1.166808  11012.955478   35628.475936  12222.500319   
min

### SUR Model + Wald Test

In [21]:
# 將教育程度轉換為虛擬變數
education_dummies = pd.get_dummies(new_df["戶長教育程度別"], prefix="戶長教育程度_dummy")
education_dummies_p = pd.get_dummies(new_df["配偶教育程度別"], prefix="配偶教育程度_dummy")

new_df = pd.concat([new_df, education_dummies], axis=1)
new_df = pd.concat([new_df, education_dummies_p], axis=1)
# 將 () 換掉，不然 python 跑不出來
new_df.rename(columns=lambda s: s
              .replace('(', '_').replace(')', '_')
              .replace('-','_').replace(' ','_'),
              inplace=True)

In [26]:
import numpy as np
import pandas as pd
from linearmodels.system import SUR

# 1. 商品類別
goods = ['主食品支出','副食品支出','乳酪及蛋類','水果類支出']

# 2. 定義所有解釋變數欄位名稱
cont_vars = ['戶內人數',"戶長年齡", '配偶年齡', '戶長所得收入', "配偶所得收入"]  # 配偶性別拿掉以避免共線性、'戶長性別'沒有變化
# 教育 dummy 的後綴
edu_levels = [
    '博士', '國_初_中_初職_', '國小', '大學',
    '專科_五專前三年劃記高職_', '碩士', '高中', '高職'
]  
# 將不識字當作 baseline 以避免共線性
# 產生戶長、配偶教育 dummy 欄位
head_edu_dummies = [f'戶長教育程度_dummy_{lv}' for lv in edu_levels]
spouse_edu_dummies = [f'配偶教育程度_dummy_{lv}' for lv in edu_levels]

# z1, z2 與總支出項目
extra = ['戶長彩券收入', '配偶彩券收入', 'log_x', 'log_x2']

# 全部 RHS 變數
regressors = cont_vars + head_edu_dummies + spouse_edu_dummies + extra
rhs = ' + '.join(regressors)

# 3. 組 equations dict
equations = {g: f"{g} ~ {rhs}" for g in goods}

In [27]:
# 4. 建 SUR 並估計
mod = SUR.from_formula(equations, new_df)
res = mod.fit(cov_type='clustered', clusters=new_df['cluster'])
print(res.summary)


                           System GLS Estimation Summary                           
Estimator:                        GLS   Overall R-squared:                   0.9077
No. Equations.:                     4   McElroy's R-squared:                 0.6872
No. Observations:                 946   Judge's (OLS) R-squared:             0.6846
Date:                Tue, Jun 10 2025   Berndt's R-squared:                  0.9177
Time:                        16:16:38   Dhrymes's R-squared:                 0.9043
                                        Cov. Estimator:                   clustered
                                        Num. Constraints:                      None
                                        Equation: 主食品支出, Dependent Variable: 主食品支出                                        
                                                        Parameter  Std. Err.     T-stat    P-value    Lower CI    Upper CI
----------------------------------------------------------------------------------

In [28]:
import itertools
import pandas as pd
from scipy.stats import chi2

# 你的 SUR 結果
params = res.params      # Series，index = ['主食品支出_戶長彩券收入', …]
cov    = res.cov         # DataFrame，同樣的 index／columns

goods = ['主食品支出','副食品支出','乳酪及蛋類','水果類支出']
z1, z2 = '戶長彩券收入', '配偶彩券收入'

tests = []
for i, j in itertools.combinations(goods, 2):
    # 1. 拼出 index 名稱
    i1 = f"{i}_{z1}"
    i2 = f"{i}_{z2}"
    j1 = f"{j}_{z1}"
    j2 = f"{j}_{z2}"
    # 2. 抽係數
    phi_i1 = params.loc[i1]
    phi_i2 = params.loc[i2]
    phi_j1 = params.loc[j1]
    phi_j2 = params.loc[j2]
    # 3. 計算 R
    R = phi_i1 * phi_j2 - phi_i2 * phi_j1

    # 4. 建梯度向量 g
    g = pd.Series(0.0, index=params.index)
    g.loc[i1] =  phi_j2
    g.loc[i2] = -phi_j1
    g.loc[j1] = -phi_i2
    g.loc[j2] =  phi_i1

    # 5. 計算 var(R) 和 Wald 統計量
    varR = float(g.dot(cov).dot(g))
    W    = R**2 / varR
    pval = chi2.sf(W, df=1)

    tests.append({
        '商品 i': i,
        '商品 j': j,
        'R': R,
        'var(R)': varR,
        'Wald': W,
        'p-value': pval
    })

# 結果整理
wald_df = pd.DataFrame(tests)
print(wald_df)


    商品 i   商品 j         R    var(R)      Wald   p-value
0  主食品支出  副食品支出  0.183135  0.335121  0.100079  0.751735
1  主食品支出  乳酪及蛋類 -0.015313  0.010514  0.022302  0.881287
2  主食品支出  水果類支出 -0.119578  0.086548  0.165214  0.684401
3  副食品支出  乳酪及蛋類 -0.018167  0.036795  0.008969  0.924548
4  副食品支出  水果類支出  0.028690  0.313219  0.002628  0.959116
5  乳酪及蛋類  水果類支出 -0.014261  0.023768  0.008556  0.926300
